# Session 1.3: BakeryAI - Prompt Engineering & Templates


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1SAIwncgxIog0VHd021vGsYPuixldyibr?usp=sharing)


## 🎯 Today's Goal

Create **professional, consistent customer interactions** using prompt templates!

### What We'll Build:

✅ Reusable prompt templates for common interactions  
✅ Order confirmation system  
✅ Structured product recommendations  
✅ Multi-language support  
✅ Automated email generation  

### Why This Matters:

Consistent communication is crucial for customer service:
- **Brand voice**: Every interaction feels professional
- **Efficiency**: Reuse templates across scenarios
- **Quality**: No more inconsistent responses
- **Scalability**: Easy to update all interactions at once

### 🚀 BakeryAI Progress: 40% → 60%
```
[████████████░░░░░░░░] 60%
```

In [1]:
# Install required packages
!pip install -q langchain langchain-openai langchain-core langchain-community
!pip install -q python-dotenv pandas

In [2]:
import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder
)
from langchain_core.output_parsers import (
    StrOutputParser,
    JsonOutputParser,
    CommaSeparatedListOutputParser
)
from langchain_core.pydantic_v1 import BaseModel, Field

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
import os
from google.colab import userdata

# Set OpenAI API key from Google Colab's user environment or default
def set_openai_api_key(default_key: str = "YOUR_API_KEY") -> None:
    """Set the OpenAI API key from Google Colab's user environment or use a default value."""
    #if not (userdata.get("OPENAI_API_KEY") or "OPENAI_API_KEY" in os.environ):
    try:
      os.environ["OPENAI_API_KEY"] = userdata.get("MDX_OPENAI_API_KEY")
    except:
      os.environ["OPENAI_API_KEY"] = default_key

set_openai_api_key()
#set_openai_api_key("sk-...")

# Verify API key is loaded
if os.getenv("OPENAI_API_KEY"):
    print("✅ OpenAI API key loaded successfully!")
else:
    print("❌ OpenAI API key not found. Please set it in .env file")

# Initialize models
llm = ChatOpenAI(model="gpt-5-nano")

✅ OpenAI API key loaded successfully!


In [4]:
!git clone https://github.com/IvanReznikov/mdx-langchain-conclave

Cloning into 'mdx-langchain-conclave'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 1), reused 19 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 239.34 KiB | 6.47 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [5]:
# Load data
try:
    cakes_df = pd.read_csv('/content/mdx-langchain-conclave/data/cake_descriptions.csv', encoding='cp1252')
    orders_df = pd.read_csv('/content/mdx-langchain-conclave/data/orders.csv', encoding='cp1252')
except FileNotFoundError:
    cakes_df = pd.DataFrame({
        'Name': ['Chocolate Truffle Cake', 'Vanilla Bean Cake', 'Red Velvet Cake'],
        'Description': ['Rich chocolate', 'Classic vanilla', 'Velvety red'],
        'Energy_kcal': [450, 380, 420],
        'Delivery_time_hr': [24, 24, 48]
    })

print("✅ Environment ready!")

✅ Environment ready!


## 1. Basic Prompt Templates for BakeryAI

Create reusable templates for common customer interactions.

In [6]:
# Product inquiry template
product_inquiry_template = ChatPromptTemplate.from_messages([
    ("system", """
    You are BakeryAI, a friendly and knowledgeable bakery assistant.

    Brand Voice:
    - Warm and welcoming
    - Professional but not stuffy
    - Enthusiastic about our products
    - Always helpful and informative

    Product: {product_name}
    Description: {product_description}
    Calories: {calories} kcal
    Delivery Time: {delivery_time} hours
    """),
    ("human", "{customer_question}")
])

# Create chain
product_chain = product_inquiry_template | llm | StrOutputParser()

# Test it
response = product_chain.invoke({
    "product_name": "Chocolate Truffle Cake",
    "product_description": "Rich chocolate cake with truffle filling",
    "calories": 450,
    "delivery_time": 24,
    "customer_question": "Is this cake suitable for a birthday party?"
})

print("🍰 BakeryAI Response:")
print(response)

🍰 BakeryAI Response:
Absolutely! The Chocolate Truffle Cake is a wonderful choice for a birthday party. Its rich chocolate cake with a silky truffle filling feels festive and indulgent, and it’s sure to be a crowd-pleaser.

Ways to make it party-ready:
- Size and servings: We offer multiple sizes to fit your guest count (tell me how many guests you’re expecting and I’ll suggest the right size).
- Birthday customization: Add a Happy Birthday message, edible toppers, and coordinating decorations or ribbons.
- Presentation: We can decorate with birthday candles, chocolate shavings, or a themed finish to match your party vibe.

Delivery and timing:
- Delivery typically takes 24 hours from order. If you have a specific date, I can help plan accordingly.

Dietary needs:
- If anyone has dietary restrictions (nuts, dairy, gluten, etc.), let me know and I’ll check which options we can accommodate.

Would you like me to check availability for your date, and get a size and decoration plan started

## 2. Order Confirmation Template

Generate professional order confirmations with structured output.

In [7]:
# Define order confirmation structure
class OrderConfirmation(BaseModel):
    order_number: str = Field(description="Unique order identifier")
    customer_name: str = Field(description="Customer's name")
    items: list = Field(description="List of ordered items")
    total_amount: float = Field(description="Total order amount")
    delivery_date: str = Field(description="Expected delivery date")
    confirmation_message: str = Field(description="Friendly confirmation message")

# JSON parser
order_parser = JsonOutputParser(pydantic_object=OrderConfirmation)

# Order confirmation template
order_template = ChatPromptTemplate.from_messages([
    ("system", """You are BakeryAI's order processing system.
    Generate a friendly order confirmation based on the provided details.
    Be warm and reassuring. Include delivery expectations."""),
    ("human", """{order_details}

    {format_instructions}""")
])

order_chain = order_template | llm | order_parser

# Test order confirmation
sample_order = """
Order #12345
Customer: Sarah Johnson
Items: 1x Chocolate Truffle Cake, 2x Vanilla Cupcakes
Total: $65.00
Delivery: Tomorrow, 2 PM
"""

confirmation = order_chain.invoke({
    "order_details": sample_order,
    "format_instructions": order_parser.get_format_instructions()
})

print("📧 Order Confirmation:")
print(confirmation)

📧 Order Confirmation:
{'order_number': '12345', 'customer_name': 'Sarah Johnson', 'items': ['1x Chocolate Truffle Cake', '2x Vanilla Cupcakes'], 'total_amount': 65.0, 'delivery_date': 'Tomorrow, 2 PM', 'confirmation_message': "Hi Sarah, your order #12345 is confirmed. We've prepared 1x Chocolate Truffle Cake and 2x Vanilla Cupcakes for you. The total is $65.00. Delivery is scheduled for tomorrow at 2 PM. Please ensure someone is available to receive the order. If you need to adjust anything, just reply and we'll be happy to help."}


## 3. Product Recommendation Template with Reasoning

In [8]:
class ProductRecommendation(BaseModel):
    recommended_product: str = Field(description="Name of recommended product")
    reasons: list = Field(description="List of reasons for recommendation")
    alternatives: list = Field(description="2 alternative products")
    price_range: str = Field(description="Expected price range")
    personalized_message: str = Field(description="Personalized recommendation message")

recommendation_parser = JsonOutputParser(pydantic_object=ProductRecommendation)

recommendation_template = ChatPromptTemplate.from_messages([
    ("system", """You are BakeryAI's recommendation engine.
    Analyze customer preferences and suggest the perfect product.

    Available Products:
    {product_catalog}

    Provide thoughtful recommendations with clear reasoning."""),
    ("human", """Customer: {customer_name}
    Occasion: {occasion}
    Preferences: {preferences}
    Budget: {budget}

    {format_instructions}""")
])

recommendation_chain = recommendation_template | llm | recommendation_parser

# Create product catalog string
product_catalog = "\n".join([
    f"- {row['Name']}: {row['Description']} ({row['Energy_kcal']} kcal)"
    for _, row in cakes_df.head(3).iterrows()
])

# Get recommendation
recommendation = recommendation_chain.invoke({
    "product_catalog": product_catalog,
    "customer_name": "Michael",
    "occasion": "Anniversary dinner",
    "preferences": "Loves chocolate, wants something elegant",
    "budget": "$50-80",
    "format_instructions": recommendation_parser.get_format_instructions()
})

print("🎯 Personalized Recommendation:")
print(f"\nRecommended: {recommendation['recommended_product']}")
print(f"\nReasons:")
for reason in recommendation['reasons']:
    print(f"  • {reason}")
print(f"\nMessage: {recommendation['personalized_message']}")

🎯 Personalized Recommendation:

Recommended: Dubai Midnight Pistachio Fantasy

Reasons:
  • Elegant presentation and name—perfect for an anniversary dinner.
  • Contains white chocolate ganache, offering a refined chocolate element that will please a chocolate lover.
  • Luxurious pistachio-forward flavor with subtle floral notes, sophisticated enough for a special occasion.
  • Priced within the requested budget of $50-$80.

Message: Hi Michael, for your anniversary dinner, Dubai Midnight Pistachio Fantasy offers an elegant centerpiece with a chocolate accent from the white ganache and a refined pistachio flavor. It fits your $50-$80 budget and pairs well with a celebratory mood. If you’d prefer something leaner or more chocolate-forward, I can explore additional options.


## 4. Few-Shot Learning for Customer Service

Train BakeryAI on example interactions.

In [9]:
# Customer service examples
cs_examples = [
    {
        "customer_message": "My order arrived damaged",
        "bakery_response": "I'm so sorry to hear that! We take great pride in our products and packaging. I'll immediately arrange for a replacement to be delivered within 24 hours at no extra cost. Can you please share your order number?"
    },
    {
        "customer_message": "Do you have vegan options?",
        "bakery_response": "Yes, we have several delicious vegan options! Our Vegan Chocolate Cake and Lemon Almond Cake are customer favorites. They're made with plant-based ingredients without compromising on taste. Would you like to know more about these options?"
    },
    {
        "customer_message": "Can I change my delivery time?",
        "bakery_response": "Of course! We understand plans change. Could you please share your order number and preferred new delivery time? I'll do my best to accommodate your request."
    }
]

# Create example template
example_template = PromptTemplate(
    input_variables=["customer_message", "bakery_response"],
    template="Customer: {customer_message}\nBakeryAI: {bakery_response}"
)

# Few-shot prompt
few_shot_prompt = FewShotPromptTemplate(
    examples=cs_examples,
    example_prompt=example_template,
    prefix="""You are BakeryAI customer service. Learn from these examples:""",
    suffix="Customer: {input}\nBakeryAI:",
    input_variables=["input"]
)

# Test few-shot learning
test_queries = [
    "I have a severe nut allergy, which cakes are safe?",
    "The cake was too sweet for my taste",
    "Do you deliver on Sundays?"
]

for query in test_queries:
    prompt = few_shot_prompt.format(input=query)
    response = llm.invoke(prompt)
    print(f"\n🙋 Customer: {query}")
    print(f"🍰 BakeryAI: {response.content}")
    print("-" * 70)


🙋 Customer: I have a severe nut allergy, which cakes are safe?
🍰 BakeryAI: I understand—nut allergies are serious, and your safety comes first. We do offer nut-free options and take precautions to prevent cross-contact, but please note that trace amounts can still occur in shared kitchens.

How can I help right now?
- I can pull up our current nut-free cake options and confirm which ones are prepared with extra precautions.
- I can check the ingredients of a specific cake you’re considering to confirm it contains no nuts.
- If this is for an existing order, share your order number and I’ll flag it as nut-free and ensure no nut-containing toppings are used.

A few quick questions to tailor my help:
- Any flavor or size preferences (vanilla, chocolate, fruit-based, etc.)?
- Are there any other allergies I should be aware of?
- Is this for a future order or an existing order? If the latter, please share the order number.

Would you like me to start by listing our nut-free options?
------

## 5. Email Generation Templates

Automated emails for various scenarios.

In [10]:
# Email templates for different scenarios
email_templates = {
    "order_confirmation": ChatPromptTemplate.from_template(
        """Subject: Order Confirmation #{order_id} - BakeryAI

Dear {customer_name},

Thank you for your order! We're excited to prepare your delicious treats.

Order Details:
{order_items}

Delivery: {delivery_date} at {delivery_time}
Total: ${total_amount}

Your order will be freshly prepared and delivered with care.

Track your order: [tracking_link]

Sweet regards,
BakeryAI Team
        """
    ),

    "delivery_reminder": ChatPromptTemplate.from_template(
        """Subject: Your BakeryAI Order Arrives Tomorrow! 🍰

Hi {customer_name},

Exciting news! Your order #{order_id} will be delivered tomorrow {delivery_date}.

Items:
{order_items}

Please ensure someone is available to receive your order.

Questions? Reply to this email or call us at [phone].

Can't wait for you to enjoy!
BakeryAI Team
        """
    ),

    "feedback_request": ChatPromptTemplate.from_template(
        """Subject: How Was Your BakeryAI Experience? 🌟

Hello {customer_name},

We hope you enjoyed your recent order of {product_name}!

Your feedback helps us improve. Would you mind taking 2 minutes to share your thoughts?

[Feedback Survey Link]

As a thank you, enjoy 10% off your next order with code: THANKYOU10

Warm wishes,
BakeryAI Team
        """
    )
}

# Generate sample emails
order_email = email_templates["order_confirmation"].format(
    order_id="12345",
    customer_name="Sarah",
    order_items="- 1x Chocolate Truffle Cake\n- 2x Vanilla Cupcakes",
    delivery_date="Friday, March 15",
    delivery_time="2:00 PM",
    total_amount="65.00"
)

print("📧 Generated Email:")
print(order_email)

📧 Generated Email:
Human: Subject: Order Confirmation #12345 - BakeryAI

Dear Sarah,

Thank you for your order! We're excited to prepare your delicious treats.

Order Details:
- 1x Chocolate Truffle Cake
- 2x Vanilla Cupcakes

Delivery: Friday, March 15 at 2:00 PM
Total: $65.00

Your order will be freshly prepared and delivered with care.

Track your order: [tracking_link]

Sweet regards,
BakeryAI Team
        


## 6. Multi-Language Support

Translate customer interactions automatically.

In [11]:
multilang_template = ChatPromptTemplate.from_messages([
    ("system", """You are BakeryAI's multilingual customer service.
    Respond in {language} with the same warm, professional tone.

    Context: {context}"""),
    ("human", "{customer_message}")
])

multilang_chain = multilang_template | llm | StrOutputParser()

# Test in different languages
languages = ["Spanish", "French", "Arabic"]
context = "We have Chocolate Truffle Cake and Vanilla Bean Cake available for delivery."
message = "What cakes do you have available?"

for lang in languages:
    response = multilang_chain.invoke({
        "language": lang,
        "context": context,
        "customer_message": message
    })
    print(f"\n[{lang}]")
    print(response)
    print("-" * 60)


[Spanish]
¡Hola! Gracias por preguntar. Tenemos dos pasteles disponibles para entrega:

- Pastel de Trufa de Chocolate
- Pastel de Vainilla

Ambos listos para entrega. ¿Qué tamaño/porciones necesitas y para qué fecha? Si me das la dirección de entrega, también puedo verificar horarios. ¿Quieres que te ayude a hacer el pedido ahora mismo?
------------------------------------------------------------

[French]
Nous avons deux gâteaux disponibles pour livraison:
- Gâteau Truffe au Chocolat
- Gâteau à la Vanille

Souhaitez-vous connaître les tailles et les prix, ou passer une commande dès maintenant ?
------------------------------------------------------------

[Arabic]
يسعدنا إخبارك بأن لدينا خيارين متاحين للتوصيل الآن:

- كعكة الترافل بالشوكولاتة — كعكة شوكولاتة غنية مع حشوة وتغطية ترافل كريمية.
- كعكة الفانيلا وبذور الفانيلا — كعكة فانيلا ناعمة مع نكهة الفانيلا وبذور الفانيلا.

هل تود معرفة الأحجام المتاحة والأسعار، أو ترغب بعمل طلب توصيل الآن؟
-----------------------------------------

## 7. Dynamic Pricing Messages

Generate pricing information with promotions.

In [12]:
def get_current_promotions():
    """Simulated promotion data"""
    return {
        "active": True,
        "discount": 15,
        "code": "SPRING15",
        "expiry": "March 31"
    }

pricing_template = PromptTemplate(
    input_variables=["product_name", "base_price"],
    template="""Product: {product_name}
    Base Price: ${base_price}

    Current Promotion: {promotion}

    Write an engaging price message that highlights the value and encourages purchase.
    Keep it under 50 words.""",
    partial_variables={"promotion": lambda: str(get_current_promotions())}
)

pricing_chain = pricing_template | llm | StrOutputParser()

price_message = pricing_chain.invoke({
    "product_name": "Chocolate Truffle Cake",
    "base_price": "45"
})

print("💰 Pricing Message:")
print(price_message)

💰 Pricing Message:
Chocolate Truffle Cake—irresistible value with 15% off using SPRING15 until March 31. Originally $45, now $38.25. Rich ganache, luxurious cocoa—perfect for any celebration. Buy now and savor the savings.


## 8. Chain-of-Thought for Complex Queries

Handle multi-step customer requests.

In [13]:
cot_template = ChatPromptTemplate.from_messages([
    ("system", """You are BakeryAI's problem-solving assistant.
    Break down complex requests step-by-step.

    Available Data:
    {data_context}

    Think through the problem:
    1. Understand what the customer needs
    2. Check what's available
    3. Provide a comprehensive solution"""),
    ("human", "{customer_request}")
])

cot_chain = cot_template | llm | StrOutputParser()

# Complex request
data_context = f"""
Products: {cakes_df[['Name', 'Delivery_time_hr']].to_string()}
Today's date: {datetime.now().strftime('%Y-%m-%d')}
"""

complex_request = """
I need a cake for tomorrow's party, but I also need to know if you can deliver
to downtown, and I'm not sure what flavor most people like. Can you help?
"""

response = cot_chain.invoke({
    "data_context": data_context,
    "customer_request": complex_request
})

print("🤔 BakeryAI's Thoughtful Response:")
print(response)

🤔 BakeryAI's Thoughtful Response:
Great idea. Here’s how we can get this sorted for tomorrow.

Step 1: Confirm the delivery area
- You mentioned “downtown.” To finalize delivery, please tell me the city (and the exact downtown area or address if you have one). We do deliver to downtown areas in many cities, but I’ll need the city to confirm.

Step 2: Pick a crowd-pleasing flavor (tomorrow-ready)
All of these flavors have quick 12-hour or shorter delivery times, so they’re suitable for a tomorrow party:
- Red Velvet Cake (12 hr)
- Cheesecake (12 hr)
- Strawberry Shortcake (12 hr)
- Chocolate Truffle Cake (12 hr)
- Black Forest Cake (12 hr)
- Tiramisu Cake (12 hr)

If you want a single flavor that tends to please most people, Red Velvet Cake, Cheesecake, or Strawberry Shortcake are strong, crowd-pleasing choices.

Step 3: Practical options (for a typical party)
Tell me your guest count (or desired size), and I’ll suggest the appropriate size. A common starting option is an 8-inch cake th

## 🎯 Exercise 5: Build a Complete Order Flow

**Task**: Create a multi-step order template system:
1. Product selection with recommendations
2. Customization options (size, message, etc.)
3. Delivery details collection
4. Order confirmation generation
5. Follow-up email scheduling

In [14]:
class OrderFlowManager:
    def __init__(self):
        self.order_data = {}

    def step1_product_selection(self, customer_preferences):
        """Recommend products based on preferences"""
        # TODO: Implement product recommendation
        pass

    def step2_customization(self, product, custom_requests):
        """Handle customization options"""
        # TODO: Process customizations
        pass

    def step3_delivery_details(self, delivery_info):
        """Collect and validate delivery information"""
        # TODO: Validate delivery details
        pass

    def step4_generate_confirmation(self):
        """Generate order confirmation"""
        # TODO: Create confirmation with all details
        pass

    def step5_schedule_followup(self, days_after=3):
        """Schedule follow-up email"""
        # TODO: Generate follow-up email content
        pass

# Test the order flow
# order_manager = OrderFlowManager()
# order_manager.step1_product_selection("chocolate lover, birthday party")

## 🎯 Exercise 6: A/B Testing Response Templates

**Task**: Create a system to test different prompt templates:
1. Define 2-3 variations of a customer service response
2. Generate responses from each template
3. Compare effectiveness (length, tone, completeness)
4. Recommend the best version

In [15]:
def ab_test_templates(scenario, template_variants):
    """
    Test multiple template variations

    Args:
        scenario: Customer service scenario
        template_variants: List of different prompt templates

    Returns:
        Analysis of each template's effectiveness
    """
    # TODO: Implement A/B testing logic
    pass

# Example scenario
# scenario = "Customer wants to cancel an order placed 2 hours ago"
# templates = [
#     "template_variant_1: formal and brief",
#     "template_variant_2: warm and detailed",
#     "template_variant_3: problem-solving focused"
# ]
# ab_test_templates(scenario, templates)

## Summary: What We Built

### ✅ Session 1.3 Achievements:

1. **Prompt Templates**: Reusable templates for consistency
2. **Order System**: Structured order confirmations
3. **Smart Recommendations**: JSON-formatted suggestions with reasoning
4. **Few-Shot Learning**: Train on example interactions
5. **Email Automation**: Professional email generation
6. **Multi-Language**: Serve global customers
7. **Complex Queries**: Chain-of-thought for problem-solving

### 🚀 BakeryAI Progress: 60%

```
[████████████░░░░░░░░] 60%
```

### Key Capabilities Added:

✨ **Consistent Brand Voice**: Every interaction feels professional  
✨ **Structured Outputs**: JSON for easy integration  
✨ **Automated Communications**: Orders, reminders, feedback  
✨ **Global Reach**: Multi-language support  
✨ **Smart Problem Solving**: Handle complex customer requests  

### Next: Notebook 1.4

We'll add **conversation memory** so BakeryAI can:
- Remember customer preferences across conversations
- Handle multi-turn order taking
- Maintain context throughout a session
- Provide personalized experiences